# Tutorial 02: Navier-Stokes problem

In this tutorial we compare the formulation of a Navier-Stokes problem using standard assembly with mixed function spaces and block assembly.

In [ ]:
import numpy as np
from mpi4py import MPI
from petsc4py import PETSc
from ufl import (derivative, div, dx, FiniteElement, grad, inner, MixedElement, split, TestFunction,
                 TrialFunction, VectorElement)
from dolfinx import DirichletBC, Function, FunctionSpace
from dolfinx.fem import (apply_lifting, assemble_matrix, assemble_matrix_block, assemble_scalar,
                         assemble_vector, assemble_vector_block, BlockVecSubVectorWrapper,
                         create_matrix, create_matrix_block, create_vector, create_vector_block,
                         locate_dofs_topological, set_bc)
from dolfinx.io import XDMFFile
from dolfinx.plotting import plot

### Constitutive parameters

In [ ]:
nu = 0.01


def u_in_eval(x):
    values = np.zeros((2, x.shape[1]))
    values[0, :] = 1.0
    return values


def u_wall_eval(x):
    return np.zeros((2, x.shape[1]))

### Mesh

In [ ]:
with XDMFFile(MPI.COMM_WORLD, "data/backward_facing_step.xdmf", "r") as infile:
    mesh = infile.read_mesh()
    mesh.create_connectivity_all()
    boundaries = infile.read_meshtags(mesh, name="boundaries")
boundaries_1 = boundaries.indices[boundaries.values == 1]
boundaries_2 = boundaries.indices[boundaries.values == 2]

In [ ]:
plot(mesh)

### Function spaces

In [ ]:
V_element = VectorElement("Lagrange", mesh.ufl_cell(), 2)
Q_element = FiniteElement("Lagrange", mesh.ufl_cell(), 1)

### Standard FEniCSx formulation using a mixed function space

In [ ]:
def run_monolithic():
    # Function spaces
    W_element = MixedElement(V_element, Q_element)
    W = FunctionSpace(mesh, W_element)

    # Test and trial functions: monolithic
    vq = TestFunction(W)
    (v, q) = split(vq)
    dup = TrialFunction(W)
    up = Function(W)
    (u, p) = split(up)

    # Variational forms
    F = (nu * inner(grad(u), grad(v)) * dx
         + inner(grad(u) * u, v) * dx
         - div(v) * p * dx
         + div(u) * q * dx)
    J = derivative(F, up, dup)

    # Boundary conditions
    u_in = Function(W.sub(0).collapse())
    u_in.interpolate(u_in_eval)
    u_wall = Function(W.sub(0).collapse())
    u_wall.interpolate(u_wall_eval)
    bdofs_V_1 = locate_dofs_topological((W.sub(0), W.sub(0).collapse()), mesh.topology.dim - 1, boundaries_1)
    bdofs_V_2 = locate_dofs_topological((W.sub(0), W.sub(0).collapse()), mesh.topology.dim - 1, boundaries_2)
    inlet_bc = DirichletBC(u_in, bdofs_V_1, W.sub(0))
    wall_bc = DirichletBC(u_wall, bdofs_V_2, W.sub(0))
    bc = [inlet_bc, wall_bc]

    # Class for interfacing with SNES
    class NavierStokesProblem(object):
        def __init__(self, F, J, solution, bcs, P=None):
            self._F = F
            self._J = J
            self._obj_vec = create_vector(F)
            self._solution = solution
            self._bcs = bcs
            self._P = P

        def update_solution(self, x):
            x.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)
            with x.localForm() as _x, self._solution.vector.localForm() as _solution:
                _solution[:] = _x

        def obj(self, snes, x):
            self.F(snes, x, self._obj_vec)
            return self._obj_vec.norm()

        def F(self, snes, x, F_vec):
            self.update_solution(x)
            with F_vec.localForm() as F_vec_local:
                F_vec_local.set(0.0)
            assemble_vector(F_vec, self._F)
            apply_lifting(F_vec, [self._J], [self._bcs], x0=[x], scale=-1.0)
            F_vec.ghostUpdate(addv=PETSc.InsertMode.ADD, mode=PETSc.ScatterMode.REVERSE)
            set_bc(F_vec, self._bcs, x, -1.0)

        def J(self, snes, x, J_mat, P_mat):
            J_mat.zeroEntries()
            assemble_matrix(J_mat, self._J, self._bcs, diagonal=1.0)
            J_mat.assemble()
            if self._P is not None:
                P_mat.zeroEntries()
                assemble_matrix(P_mat, self._P, self._bcs, diagonal=1.0)
                P_mat.assemble()

    # Create problem
    problem = NavierStokesProblem(F, J, up, bc)
    F_vec = create_vector(F)
    J_mat = create_matrix(J)

    # Solve
    snes = PETSc.SNES().create(mesh.mpi_comm())
    snes.setTolerances(max_it=20)
    snes.getKSP().setType("preonly")
    snes.getKSP().getPC().setType("lu")
    snes.getKSP().getPC().setFactorSolverType("mumps")
    snes.setObjective(problem.obj)
    snes.setFunction(problem.F, F_vec)
    snes.setJacobian(problem.J, J=J_mat, P=None)
    snes.setMonitor(lambda _, it, residual: print(it, residual))
    up_copy = up.vector.copy()
    snes.solve(None, up_copy)
    problem.update_solution(up_copy)  # TODO can this be safely removed?
    return up

In [ ]:
up_m = run_monolithic()
(u_m, p_m) = up_m.split()

In [ ]:
plot(u_m)

In [ ]:
plot(p_m)

### Block FEniCSx formulation using a two independent function spaces

In [ ]:
def run_block():
    # Function spaces
    V = FunctionSpace(mesh, V_element)
    Q = FunctionSpace(mesh, Q_element)

    # Test and trial functions
    (v, q) = (TestFunction(V), TestFunction(Q))
    (du, dp) = (TrialFunction(V), TrialFunction(Q))
    (u, p) = (Function(V), Function(Q))

    # Variational forms
    F = [nu * inner(grad(u), grad(v)) * dx + inner(grad(u) * u, v) * dx - div(v) * p * dx,
         div(u) * q * dx]
    J = [[derivative(F[0], u, du), derivative(F[0], p, dp)],
         [derivative(F[1], u, du), derivative(F[1], p, dp)]]

    # Boundary conditions
    u_in = Function(V)
    u_in.interpolate(u_in_eval)
    u_wall = Function(V)
    u_wall.interpolate(u_wall_eval)
    bdofs_V_1 = locate_dofs_topological(V, mesh.topology.dim - 1, boundaries_1)
    bdofs_V_2 = locate_dofs_topological(V, mesh.topology.dim - 1, boundaries_2)
    inlet_bc = DirichletBC(u_in, bdofs_V_1)
    wall_bc = DirichletBC(u_wall, bdofs_V_2)
    bc = [inlet_bc, wall_bc]

    # Class for interfacing with SNES
    class NavierStokesProblem(object):
        def __init__(self, F, J, solutions, bcs, P=None):
            self._F = F
            self._J = J
            self._obj_vec = create_vector_block(F)
            self._solutions = solutions
            self._bcs = bcs
            self._P = P

        def update_solutions(self, x):
            x.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)
            with BlockVecSubVectorWrapper(x, [V.dofmap, Q.dofmap]) as x_wrapper:
                for x_wrapper_local, sub_solution in zip(x_wrapper, self._solutions):
                    with sub_solution.vector.localForm() as sub_solution_local:
                        sub_solution_local[:] = x_wrapper_local

        def obj(self, snes, x):
            self.F(snes, x, self._obj_vec)
            return self._obj_vec.norm()

        def F(self, snes, x, F_vec):
            self.update_solutions(x)
            with F_vec.localForm() as F_vec_local:
                F_vec_local.set(0.0)
            assemble_vector_block(F_vec, self._F, self._J, self._bcs, x0=x, scale=-1.0)

        def J(self, snes, x, J_mat, P_mat):
            J_mat.zeroEntries()
            assemble_matrix_block(J_mat, self._J, self._bcs, diagonal=1.0)
            J_mat.assemble()
            if self._P is not None:
                P_mat.zeroEntries()
                assemble_matrix_block(P_mat, self._P, self._bcs, diagonal=1.0)
                P_mat.assemble()

    # Create problem
    problem = NavierStokesProblem(F, J, (u, p), bc)
    F_vec = create_vector_block(F)
    J_mat = create_matrix_block(J)

    # Solve
    snes = PETSc.SNES().create(mesh.mpi_comm())
    snes.setTolerances(max_it=20)
    snes.getKSP().setType("preonly")
    snes.getKSP().getPC().setType("lu")
    snes.getKSP().getPC().setFactorSolverType("mumps")
    snes.setObjective(problem.obj)
    snes.setFunction(problem.F, F_vec)
    snes.setJacobian(problem.J, J=J_mat, P=None)
    snes.setMonitor(lambda _, it, residual: print(it, residual))
    solution = create_vector_block(F)
    snes.solve(None, solution)
    problem.update_solutions(solution)  # TODO can this be safely removed?
    return (u, p)

In [ ]:
(u_b, p_b) = run_block()

In [ ]:
plot(u_b)

In [ ]:
plot(p_b)

### Error computation between mixed and block cases

In [ ]:
def run_error(u_m, p_m, u_b, p_b):
    u_m_norm = np.sqrt(mesh.mpi_comm().allreduce(assemble_scalar(inner(grad(u_m), grad(u_m)) * dx), op=MPI.SUM))
    err_u_norm = np.sqrt(
        mesh.mpi_comm().allreduce(assemble_scalar(inner(grad(u_b - u_m), grad(u_b - u_m)) * dx), op=MPI.SUM))
    p_m_norm = np.sqrt(mesh.mpi_comm().allreduce(assemble_scalar(inner(p_m, p_m) * dx), op=MPI.SUM))
    err_p_norm = np.sqrt(
        mesh.mpi_comm().allreduce(assemble_scalar(inner(p_b - p_m, p_b - p_m) * dx), op=MPI.SUM))
    print("Relative error for velocity component is equal to", err_u_norm / u_m_norm)
    print("Relative error for pressure component is equal to", err_p_norm / p_m_norm)
    assert np.isclose(err_u_norm / u_m_norm, 0., atol=1.e-10)
    assert np.isclose(err_p_norm / p_m_norm, 0., atol=1.e-10)

In [ ]:
run_error(u_m, p_m, u_b, p_b)